# Utiliser l'API Google Books pour récupérer des données de livres

## Partie 1 : Comprendre les API

### a. Qu’est-ce qu’une API ?
- API = Application Programming Interface
- Une API est une interface qui permet à un programme d’accéder aux fonctionnalités ou aux données d’un autre service, sans avoir besoin de connaître son fonctionnement interne.
- Exemple : l’API Google Books met à disposition des données sur des millions de livres, que l’on peut interroger depuis un programme Python.
- On envoie une **requête** (question) à l’API, elle nous répond avec des **données** (souvent en JSON)

### b. Qu’est-ce qu’une requête HTTP ?
- HTTP = HyperText Transfer Protocol – c’est le protocole qui permet à des machines (navigateur, script Python, application mobile...) de communiquer avec des serveurs sur Internet.
- Une requête HTTP est un message envoyé depuis un client (comme un navigateur ou un programme Python) vers un serveur web pour lui demander quelque chose.
- Quand on ouvre un site web ou que l'on demande des données à une API, on envoie une **requête HTTP**, généralement de type :

    - **GET** → pour demander des données
    - **POST** → pour envoyer des données
    - **PUT**, **DELETE**, etc.

- L’API Google Books utilise principalement des requêtes de type GET, comme si on entrait une URL dans un navigateur pour demander des infos.
- Une URL de requête API suit une structure bien définie :

`<URL de base> ? <paramètre1>=<valeur1> & <paramètre2>=<valeur2> & ...`

- URL de base : adresse du point d’entrée de l’API, aussi appelée **endpoint**.
- ? : sépare l’URL de base des paramètres de requête.
- Paramètres de requête (query parameters) : ajoutés sous forme de paires clé=valeur. Chaque paramètre est séparé par &.  Les paramètres possibles sont disponibles dans la documentation de chaque API.

### c. Réponse

Quand on envoie une requête GET à une API, on demande au serveur de renvoyer une ressource.

La réponse de ce serveur comporte deux parties essentielles :


1. **Le code de status de réponse (HTTP status code)**

C’est un nombre à 3 chiffres qui indique si la requête a réussi ou non.

--> [Liste des codes possibles](https://developer.mozilla.org/fr/docs/Web/HTTP/Reference/Status) et leur signification


2. **Le contenu de la réponse (le "corps")**

C’est ici que se trouvent les données demandées.

Ce contenu est souvent au format JSON (JavaScript Object Notation), c’est-à-dire une structure clé-valeur que Python peut facilement lire.

Exemple d'un format JSON :
```json
{
  "items": [
    {
      "tripInfo": {
        "country": "France",
        "cities": ["Montpellier", "Marseille"],
        "nb_photo_taken": 200,
        "language": "fr"
      }
    }
  ]
}
```

---
## Partie 2 : Requêter Google Books API

Dans cette partie vous allez : 
- Faire une requête HTTP pour interroger l'API Google Books à l'aide de la bibliothèque python _requests_,
- Vérifier le code de status de la réponse,
- Traiter le corps de la réponse à l'aide de la bibliothèque python _json_,
- Utiliser _pandas_ pour convertir les données reçues en dataframe.


In [1]:
# Importer les bibliothèques nécessaires
import requests
import json
import pandas as pd

Pour interroger une API, un certain nombre de paramètres et filtres définis dans la documentation peuvent être utilisés. 

En utilisant la [documentation](https://developers.google.com/books/docs/v1/using?hl=fr#WorkingVolumes) de l'API Google Books, définir le dictionnaire de paramètres permettant de faire une recherche sur :
- la chaîne de texte : "food",
- il doit s'agir d'un e-book avec un prix d'achat,
- le nombre maximal de résultats à renvoyer est de 40,
- les résultats doivent être triés par pertinence.

In [2]:
# URL de l'API Google Books
url = "https://www.googleapis.com/books/v1/volumes"

# Dictionnaire de paramètres pour la requête
params = {
    'q': 'food',                      # Terme de recherche
    'filter': 'ebooks',              # Uniquement les ebooks (inclut ceux en vente)
    'printType': 'books',            # Limiter aux livres (exclut magazines)
    'maxResults': 40,                # Nombre maximum de résultats
    'orderBy': 'relevance'           # Trier par pertinence
}


# Dictionnaire de paramètres pour la requête


En utilisant la documentation de la bibliothèque [_requests_](https://requests.readthedocs.io/en/latest/user/quickstart/):
- implémenter la requête HTTP _get_  avec les paramètres sur l'URL définie ci-dessus,
- vérifier le code de réponse HTTP.

In [4]:
# Requêter l'API Google Books
response = requests.get(url, params=params)

# Vérifier le code de statut de la réponse
if response.status_code == 200:
    print("✅ Requête réussie !")
else:
    print(f"❌ Échec de la requête. Code de statut : {response.status_code}")




✅ Requête réussie !


Récupérer et observer le coeur de la réponse via l'objet _response_ créé précédemment.

In [5]:
# Récupérer le coeur de la réponse
data_books_raw = response.json()

# Afficher les données récupérées
print(json.dumps(data_books_raw, indent=2))  # formaté pour lecture facile

# Afficher le type de la variable data_books_raw
print(type(data_books_raw))  # devrait être <class 'dict'>


{
  "kind": "books#volumes",
  "totalItems": 1000000,
  "items": [
    {
      "kind": "books#volume",
      "id": "sa6pDAAAQBAJ",
      "etag": "fKAZqlII0rk",
      "selfLink": "https://www.googleapis.com/books/v1/volumes/sa6pDAAAQBAJ",
      "volumeInfo": {
        "title": "Food",
        "subtitle": "A Culinary History",
        "authors": [
          "Jean-Louis Flandrin",
          "Massimo Montanari"
        ],
        "publisher": "Columbia University Press",
        "publishedDate": "1999-11-23",
        "description": "When did we first serve meals at regular hours? Why did we begin using individual plates and utensils to eat? When did \"cuisine\" become a concept and how did we come to judge food by its method of preparation, manner of consumption, and gastronomic merit? Food: A Culinary History explores culinary evolution and eating habits from prehistoric times to the present, offering surprising insights into our social and agricultural practices, religious beliefs, and m

- Les données relatives aux livres récupérées sont accessibles dans la clé 'items',
- Récupérer les données relatives aux livres dans la variable data_books.

**NOTE**

Il existe 2 méthodes pour accéder aux valeurs d'une clé ("nom_de_la_clé") d'un dictionnaire python : 
- `mon_dict["nom_de_la_clé"]`: si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela retourne une erreur.
- `mon_dict.get("nom_de_la_clé")` : si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela renvoie None par défault. 
Il est aussi possible d'indiquer comme second argument la variable à renvoyer si la clé n'est pas présente : 

`mon_dict.get("nom_de_la_clé", "clé inexistante")`

Privilégier la seconde méthode car certaines clés peuvent ne pas exister.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_dictionary_items) : accès au clé/valeurs d'un dictionnaire python

In [6]:
# Récupérer les données relatives aux livres via la clé 'items'
data_books = data_books_raw.get('items', [])

# Afficher le type de la variable data_books
print(type(data_books))  # devrait être <class 'list'>

<class 'list'>


Accéder au premier élément de _data_books_ pour afficher les données du premier livre.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_list_items) : accéder aux éléments d'une liste python

In [8]:
# Afficher le premier livre
first_book = data_books[0]



Dans les données du premier livre, chercher où se trouve les informations suivantes (il peut s'agir de dictionnaires imbriqués) :
- le titre,
- le prix,
- la note (_averageRating_)

Afficher ces informations (cf la note ci-dessus pour accéder aux éléments d'un dictionnaire en python).

In [9]:
title_first_book = data_books[0]
price_first_book = first_book.get("saleInfo", {}).get("retailPrice", {}).get("amount", "Prix non disponible")
average_rating_first_book = first_book.get("volumeInfo", {}).get("averageRating", "Note non disponible")
print(f"Titre : {title_first_book}")
print(f"Prix : {price_first_book}")
print(f"Note : {average_rating_first_book}")


Titre : {'kind': 'books#volume', 'id': 'sa6pDAAAQBAJ', 'etag': 'fKAZqlII0rk', 'selfLink': 'https://www.googleapis.com/books/v1/volumes/sa6pDAAAQBAJ', 'volumeInfo': {'title': 'Food', 'subtitle': 'A Culinary History', 'authors': ['Jean-Louis Flandrin', 'Massimo Montanari'], 'publisher': 'Columbia University Press', 'publishedDate': '1999-11-23', 'description': 'When did we first serve meals at regular hours? Why did we begin using individual plates and utensils to eat? When did "cuisine" become a concept and how did we come to judge food by its method of preparation, manner of consumption, and gastronomic merit? Food: A Culinary History explores culinary evolution and eating habits from prehistoric times to the present, offering surprising insights into our social and agricultural practices, religious beliefs, and most unreflected habits. The volume dispels myths such as the tale that Marco Polo brought pasta to Europe from China, that the original recipe for chocolate contained chili in

In [10]:
# Afficher les infos clés (price, title, rating) pour chaque livres de la liste data_books
for i, book in enumerate(data_books, start=1):
    title = book.get("volumeInfo", {}).get("title", "Titre non disponible")
    price = book.get("saleInfo", {}).get("retailPrice", {}).get("amount", "Prix non disponible")
    rating = book.get("volumeInfo", {}).get("averageRating", "Note non disponible")
    
    print(f" Livre {i}")
    print(f"  Titre  : {title}")
    print(f"  Prix   : {price}")
    print(f"  Note   : {rating}")
    print("-" * 40)

 Livre 1
  Titre  : Food
  Prix   : 17.49
  Note   : Note non disponible
----------------------------------------
 Livre 2
  Titre  : French Food France Food
  Prix   : 8.91
  Note   : Note non disponible
----------------------------------------
 Livre 3
  Titre  : A History of Food
  Prix   : 28.99
  Note   : Note non disponible
----------------------------------------
 Livre 4
  Titre  : On Food and Cooking
  Prix   : Prix non disponible
  Note   : 4
----------------------------------------
 Livre 5
  Titre  : Introduction to Food Engineering
  Prix   : 50.71
  Note   : 5
----------------------------------------
 Livre 6
  Titre  : The Philosophy of Food
  Prix   : 26.82
  Note   : Note non disponible
----------------------------------------
 Livre 7
  Titre  : Food, Energy, and Society
  Prix   : 49.15
  Note   : 4.5
----------------------------------------
 Livre 8
  Titre  : Proteomics for Food Authentication
  Prix   : 182.52
  Note   : Note non disponible
-----------------------

Créer une liste de dictionnaire contenant les informations de chaque livre sous le format : 
```
[{
    "title": "title0",
    "price": price_0,
    "rating": rating_0
},
{
    "title": "title1",
    "price": price_1,
    "rating": rating_1
},
...
]
```

In [11]:
# Création d'une liste de dictionnaires pour les livres
books_list = []
for book in data_books:
    book_info = {
        'title': book.get("volumeInfo", {}).get("title", "Titre non disponible"),
        'price': book.get("saleInfo", {}).get("retailPrice", {}).get("amount", "Prix non disponible"),
        'rating': book.get("volumeInfo", {}).get("averageRating", "Note non disponible")
    }
    books_list.append(book_info)

In [12]:
# Créer un dataframe à partir de la liste de dictionnaires
books_df = pd.DataFrame(books_list)
# Afficher les 5 premières lignes du dataframe
print(books_df.head())



                              title                price               rating
0                              Food                17.49  Note non disponible
1           French Food France Food                 8.91  Note non disponible
2                 A History of Food                28.99  Note non disponible
3               On Food and Cooking  Prix non disponible                    4
4  Introduction to Food Engineering                50.71                    5


Filtrer les livres pour ne conserver que ceux ayant des valeurs pour les colonnes _price_ et _rating_.

- Possibilité d'utiliser la méthode pandas [_dropna_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- Afficher le dataframe et regarder ce qu'il s'est passé au niveau des index.
- Utiliser la méthode [_reset_index_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) pour réinitialiser les index.

In [ ]:
# Filter les livres pour ne conserver que ceux ayant des valeurs pour les colonnes price et rating
books_df = books_df.dropna(subset=['price', 'rating'])

# Reinitialiser les index du DataFrame
books_df.reset_index(drop=True, inplace=True)

# Ajouter une colonne availability = False
books_df['availability'] = False
# Afficher le DataFrame final
print(books_df)


NameError: name 'books_df' is not defined

---
## Partie 3 : Insertion des données en base

In [14]:
# Utiliser sqlite3 pour insérer les données dans la base de données book_store dans la table book
import sqlite3
conn = sqlite3.connect('book_store.db')
books_df.to_sql('book', conn, if_exists='replace', index=False) 


40

In [15]:
# Vérifier que le nombre de livres dans la table correspond au nb de livres scrapés + livres de l'API
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM book")
count = cursor.fetchone()[0]
print(f"Nombre de livres dans la table : {count}")


Nombre de livres dans la table : 40
